<a href="https://colab.research.google.com/github/gummadhav/Let_us_Learn/blob/main/Scalable_CUDA_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y libopenmpi-dev openmpi-bin openmpi-common
!pip install opencv-python
!mkdir -p input_images output_images

# Generate some dummy .bmp images for testing
import cv2
import numpy as np
for i in range(4):
    img = np.random.randint(0, 256, (128,128), dtype=np.uint8)
    cv2.imwrite(f'input_images/test_{i}.bmp', img)


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,782 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,576 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restrict

In [2]:
%%shell
%%shell
# Update package lists
apt-get update

# Install essential build tools (if not already present, good practice)
apt-get install -y build-essential

# Install OpenCV development libraries (C++ headers and shared libraries)
# 'libopencv-dev' provides the development files
# 'python3-opencv' is for Python, but sometimes pulled in as a dependency
apt-get install -y libopencv-dev python3-opencv

# Verify OpenCV version (optional, but good for checking)
pkg-config --modversion opencv4 # For OpenCV 4.x
# Or for older versions: pkg-config --modversion opencv

/bin/bash: line 1: fg: no job control
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (source

In [7]:
!pip install nvcc4jupyter

In [8]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp0cemb41w".


In [9]:
%%shell
nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [10]:
%%shell
nvidia-smi

Sat Aug 30 17:42:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
%%writefile scalable_cuda_streams.cu

#include <mpi.h>
#include <opencv2/opencv.hpp>
#include <vector>
#include <string>
#include <filesystem>
#include <iostream>
#include "cuda_runtime.h"

#define CHECK_CUDA(call) \
    { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error: " << cudaGetErrorString(err) << " at " << __FILE__ << ":" << __LINE__ << std::endl; \
            MPI_Abort(MPI_COMM_WORLD, -1); \
        } \
    }

__global__
void gaussian_filter_kernel(const uchar* input, uchar* output, int width, int height, const float* kernel) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    float sum = 0, kSum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            float k = kernel[(dy+1)*3 + (dx+1)];
            sum += input[iy*width + ix] * k;
            kSum += k;
        }
    }
    float result = kSum > 0 ? sum / kSum : 0;
    result = min(max(result, 0.f), 255.f);
    output[y*width + x] = static_cast<uchar>(result);
}

__global__
void laplacian_filter_kernel(const uchar* input, uchar* output, int width, int height) {
    int kernel[3][3] = { {-1,-1,-1}, {-1,8,-1}, {-1,-1,-1} };
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    int sum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            sum += input[iy*width + ix] * kernel[dy+1][dx+1];
        }
    }
    int result = sum + 128;
    result = min(max(result, 0), 255);
    output[y*width + x] = static_cast<uchar>(result);
}

__global__ void empty_kernel(int *p) { *p = 1; }

std::vector<std::string> get_image_paths(const std::string& folder) {
    std::vector<std::string> paths;
    for (const auto& entry : std::filesystem::directory_iterator(folder)) {
        if (entry.path().extension() == ".bmp") {
            paths.push_back(entry.path().string());
        }
    }
    return paths;
}

void warm_up_cuda() {
    int* d_temp;
    CHECK_CUDA(cudaMalloc(&d_temp, sizeof(int)));
    empty_kernel<<<1, 1>>>(d_temp);
    CHECK_CUDA(cudaDeviceSynchronize());
    CHECK_CUDA(cudaFree(d_temp));
}

void process_images_on_rank(const std::vector<std::string>& images, const float* gaussKernel, const std::string& output_folder, int rank) {
    for (const auto& path : images) {
        cv::Mat img = cv::imread(path, cv::IMREAD_GRAYSCALE);
        if (img.empty() || img.type() != CV_8U || img.channels() != 1) {
            std::cerr << "Rank " << rank << ": Invalid image: " << path << std::endl;
            continue;
        }

        int width = img.cols, height = img.rows;
        size_t size = width * height * sizeof(uchar);

        uchar *d_input, *d_gauss, *d_laplace;
        float *d_gkernel;
        CHECK_CUDA(cudaMalloc(&d_input, size));
        CHECK_CUDA(cudaMalloc(&d_gauss, size));
        CHECK_CUDA(cudaMalloc(&d_laplace, size));
        CHECK_CUDA(cudaMalloc(&d_gkernel, 9 * sizeof(float)));

        cudaStream_t stream;
        cudaEvent_t start_event, stop_event;
        CHECK_CUDA(cudaStreamCreate(&stream));
        CHECK_CUDA(cudaEventCreate(&start_event));
        CHECK_CUDA(cudaEventCreate(&stop_event));

        if (path == images.front()) {
            warm_up_cuda();
        }

        CHECK_CUDA(cudaMemcpyAsync(d_input, img.data, size, cudaMemcpyHostToDevice, stream));
        CHECK_CUDA(cudaMemcpyAsync(d_gkernel, gaussKernel, 9 * sizeof(float), cudaMemcpyHostToDevice, stream));

        dim3 block(16,16), grid((width+15)/16, (height+15)/16);

        CHECK_CUDA(cudaEventRecord(start_event, stream));

        gaussian_filter_kernel<<<grid, block, 0, stream>>>(d_input, d_gauss, width, height, d_gkernel);
        laplacian_filter_kernel<<<grid, block, 0, stream>>>(d_gauss, d_laplace, width, height);

        CHECK_CUDA(cudaEventRecord(stop_event, stream));

        cv::Mat out(height, width, CV_8U);
        CHECK_CUDA(cudaMemcpyAsync(out.data, d_laplace, size, cudaMemcpyDeviceToHost, stream));
        CHECK_CUDA(cudaStreamSynchronize(stream));

        float elapsed_ms = 0;
        CHECK_CUDA(cudaEventElapsedTime(&elapsed_ms, start_event, stop_event));

        std::string gpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_gpu_rank" + std::to_string(rank) + ".bmp";
        if (!cv::imwrite(gpu_output, out)) {
            std::cerr << "Rank " << rank << ": Failed to save GPU image: " << gpu_output << std::endl;
        }

        std::cout << "Rank " << rank << ": Processed " << path << " in " << elapsed_ms << " ms\n";

        CHECK_CUDA(cudaFree(d_input));
        CHECK_CUDA(cudaFree(d_gauss));
        CHECK_CUDA(cudaFree(d_laplace));
        CHECK_CUDA(cudaFree(d_gkernel));
        CHECK_CUDA(cudaEventDestroy(start_event));
        CHECK_CUDA(cudaEventDestroy(stop_event));
        CHECK_CUDA(cudaStreamDestroy(stream));
    }
}

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    std::string input_folder = "input_images";
    std::string output_folder = "output_images";

    if (rank == 0) {
        std::filesystem::create_directories(output_folder);
    }
    MPI_Barrier(MPI_COMM_WORLD);

    auto all_paths = get_image_paths(input_folder);

    std::vector<std::string> images_for_rank;
    for (size_t i = rank; i < all_paths.size(); i += size) {
        images_for_rank.push_back(all_paths[i]);
    }

    int device_count = 0;
    CHECK_CUDA(cudaGetDeviceCount(&device_count));
    if (device_count == 0) {
        if (rank == 0) std::cerr << "No CUDA devices found.\n";
        MPI_Finalize();
        return -1;
    }
    int device_id = 0; // Colab only gives you one GPU, so all ranks use device 0
    CHECK_CUDA(cudaSetDevice(device_id));
    if (rank == 0)
        std::cout << "MPI ranks: " << size << ", each using GPU " << device_id << std::endl;

    float gaussKernel[9] = {1.f/16, 2.f/16, 1.f/16, 2.f/16, 4.f/16, 2.f/16, 1.f/16, 2.f/16, 1.f/16};

    process_images_on_rank(images_for_rank, gaussKernel, output_folder, rank);

    MPI_Finalize();
    return 0;
}


Writing scalable_cuda_streams.cu


In [19]:
%%shell

nvcc scalable_cuda_streams.cu -o scalable_cuda_streams -I/usr/lib/x86_64-linux-gnu/openmpi/include -L/usr/lib/x86_64-linux-gnu/openmpi/lib -lmpi -lstdc++fs -std=c++17 $(pkg-config --cflags --libs opencv4) -arch=sm_75

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [20]:
%%shell
ls -l scalable_cuda_streams

-rwxr-xr-x 1 root root 1087816 Aug 30 17:53 scalable_cuda_streams


In [25]:
%%shell

mpirun --allow-run-as-root --oversubscribe -np 4 ./scalable_cuda_streams

Rank 2: Processed input_images/test_0.bmp in 0.06304 ms
MPI ranks: 4, each using GPU 0
Rank 1: Processed input_images/test_3.bmp in 0.349728 ms
Rank 3: Processed input_images/test_1.bmp in 0.024032 ms
Rank 0: Processed input_images/test_2.bmp in 0.024608 ms
